# Imports

In [1]:
from ansatz import Ansatz

In [2]:
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

Device: cuda


In [4]:
my_ansatz = Ansatz(n_qubits = 2, n_layers = 1)

# Architecture definition

In [5]:
class SequentialLinearModule(nn.Module):

    def __init__(self, in_dim, out_dim, layer_sizes, activation, out_activation):
        
        '''
        layers: list containing the size of the linear layers between input and output
        '''

        super().__init__()
        self.out_dim = out_dim
        
        # add first and last layers
        layer_sizes.insert(0, in_dim)
        layer_sizes.append(out_dim*2)
        
        num_layers = len(layer_sizes)-1
        layers_list = []
        for i in range(num_layers):
            if i!=0:
                layers_list.append(activation())
            layers_list.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            # layers_list.append(nn.BatchNorm1d(layer_sizes[i+1]))
            # for some reason batchnorm ruins everything, maybe add on the first few layers?
        layers_list.append(out_activation())
        
        self.f = nn.Sequential(*layers_list)
        
    def forward(self, x):
        sin_theta, cos_theta = torch.split(self.f(x), self.out_dim, 1)
        
        return torch.atan2(sin_theta, cos_theta)

# Dataset definition

In [6]:
class AnsatzDataset(torch.utils.data.TensorDataset):
    def __init__(self, n_pairs):
        self.psi, self.theta = my_ansatz.generate_pairs(n_pairs)
        
    def __getitem__(self, idx):
        return self.psi[idx], self.theta[idx]
    
    def __len__(self):
        return len(self.psi)

In [7]:
train_data = AnsatzDataset(1024*100)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=512)

test_data = AnsatzDataset(1024)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=512)

# Initializing model

In [10]:
in_dim = 2**my_ansatz.n_qubits
out_dim = my_ansatz.n_theta
out_activation = nn.Tanh
activation = nn.LeakyReLU
layer_sizes = [(in_dim+out_dim)*2+40]*8
lr = 3e-4

model = SequentialLinearModule(in_dim, out_dim, layer_sizes, activation, out_activation).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = nn.MSELoss(reduction='sum')

In [11]:
print(model)

SequentialLinearModule(
  (f): Sequential(
    (0): Linear(in_features=4, out_features=56, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=56, out_features=56, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=56, out_features=56, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=56, out_features=56, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=56, out_features=56, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): Linear(in_features=56, out_features=56, bias=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=56, out_features=56, bias=True)
    (13): LeakyReLU(negative_slope=0.01)
    (14): Linear(in_features=56, out_features=56, bias=True)
    (15): LeakyReLU(negative_slope=0.01)
    (16): Linear(in_features=56, out_features=8, bias=True)
    (17): Tanh()
  )
)


# Train loop

In [12]:
def train(n_epochs):
    for i in range(n_epochs):
        running_loss = 0
        model.train()
        for j, (psi_true, theta_true) in enumerate(train_dataloader):
            
            theta_true = theta_true.type(torch.cuda.FloatTensor).to(device)
            psi_true = psi_true.type(torch.cuda.FloatTensor).to(device)
            theta_pred = model(psi_true)
            loss = loss_function(theta_pred, theta_true)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        if not i%10:
            print(f'epoch {i}, training loss: {running_loss/len(train_data)}')

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_function = nn.MSELoss(reduction='sum')
train(10000)

epoch 0, training loss: 15.648360424041748
epoch 10, training loss: 7.457572538852691
epoch 20, training loss: 6.914449343681335
epoch 30, training loss: 8.14009210586548
epoch 40, training loss: 6.811366710662842
epoch 50, training loss: 8.138894951343536
epoch 60, training loss: 6.585956060886383
epoch 70, training loss: 7.3378882455825805
epoch 80, training loss: 8.699550716876983
epoch 90, training loss: 6.757590854167939
epoch 100, training loss: 7.280844666957855
epoch 110, training loss: 7.3591417932510375
epoch 120, training loss: 7.3494776082038875
epoch 130, training loss: 6.708052275180816
epoch 140, training loss: 6.694174191951752
epoch 150, training loss: 6.6840537166595455
epoch 160, training loss: 7.355789473056793
epoch 170, training loss: 8.832986562252044
epoch 180, training loss: 8.413821928501129
epoch 190, training loss: 7.686001582145691
epoch 200, training loss: 6.738543925285339
epoch 210, training loss: 7.870853586196899
epoch 220, training loss: 7.63351154565

epoch 1850, training loss: 7.5508950066566465
epoch 1860, training loss: 8.476918623447418
epoch 1870, training loss: 7.663112421035766
epoch 1880, training loss: 7.665670683383942
epoch 1890, training loss: 7.2890850043296815
epoch 1900, training loss: 7.479550640583039
epoch 1910, training loss: 7.914831972122192
epoch 1920, training loss: 8.352455449104308
epoch 1930, training loss: 7.433028371334076
epoch 1940, training loss: 7.6153326511383055
epoch 1950, training loss: 7.4616093873977665
epoch 1960, training loss: 6.816531982421875
epoch 1970, training loss: 7.490635540485382
epoch 1980, training loss: 6.846096849441528
epoch 1990, training loss: 7.1934924101829525
epoch 2000, training loss: 8.187871692180634
epoch 2010, training loss: 8.738523564338685
epoch 2020, training loss: 8.020571203231812
epoch 2030, training loss: 7.520224890708923
epoch 2040, training loss: 8.9087965798378
epoch 2050, training loss: 8.227407355308532
epoch 2060, training loss: 8.938180799484252
epoch 2

epoch 3680, training loss: 6.895452461242676
epoch 3690, training loss: 6.815780580043793
epoch 3700, training loss: 6.97844144821167
epoch 3710, training loss: 7.070657095909119
epoch 3720, training loss: 6.748726511001587
epoch 3730, training loss: 6.711070663928986
epoch 3740, training loss: 8.303292846679687
epoch 3750, training loss: 7.179029858112335
epoch 3760, training loss: 6.711044554710388
epoch 3770, training loss: 6.721805703639984
epoch 3780, training loss: 6.739846451282501
epoch 3790, training loss: 7.007671463489532
epoch 3800, training loss: 6.674271309375763
epoch 3810, training loss: 6.831736817359924
epoch 3820, training loss: 7.2285942721366885
epoch 3830, training loss: 7.64167130947113
epoch 3840, training loss: 6.92780202627182
epoch 3850, training loss: 7.399382064342499
epoch 3860, training loss: 7.961297125816345
epoch 3870, training loss: 6.995720822811126
epoch 3880, training loss: 6.964945685863495
epoch 3890, training loss: 7.0157902050018315
epoch 3900,

KeyboardInterrupt: 

# Manual testing

In [16]:
model.eval()

pair = my_ansatz.generate_pairs(1)
psi_true = torch.FloatTensor(pair[0])
theta_true = torch.FloatTensor(pair[1])

#psi_true, theta_true = next(iter(train_dataloader))

theta_true = theta_true.type(torch.cuda.FloatTensor).to(device)
psi_true = psi_true.type(torch.cuda.FloatTensor).to(device)

theta_pred = model(psi_true)
psi_pred = my_ansatz.apply_ansatz(theta_pred.cpu().detach().numpy()[0])

psi_true = psi_true.cpu().detach().numpy()[0]
theta_pred = theta_pred.cpu().detach().numpy()[0]
theta_true = theta_true.cpu().detach().numpy()[0]

In [17]:
theta_true

array([ 0.87727344, -1.9147718 , -1.3180441 ,  2.3590467 ], dtype=float32)

In [18]:
theta_pred

array([ 0.7401611 , -0.78566843, -0.00961818,  0.00991248], dtype=float32)

In [19]:
psi_true

array([ 0.5211627 , -0.34727734, -0.7402801 ,  0.24448584], dtype=float32)

In [20]:
psi_pred

array([ 0.86128301, -0.13845813, -0.35689147,  0.33413978])